In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

from pathlib import Path
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import json


from IPython.display import display
import pandas_profiling
from pandas_profiling import ProfileReport
%matplotlib inline

from lightfm import LightFM



print(os.path.abspath(''))
root_path = os.path.abspath('')+"/../"
print(root_path)
data_folder = root_path+"/data/"

C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks
C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks/../


C:\Users\samir\anaconda3\envs\retail-rocket\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
### read users interaction
events = pd.read_csv(data_folder+"events.csv", header= 0)
events_reduced = events.drop_duplicates()
events_reduced = events_reduced.drop_duplicates(['visitorid', 'itemid'], keep='last').copy()
events_tupel = []
events_final = events_reduced.sort_values(by='timestamp', ascending=True).copy()
events_final.loc[events_final['event'] == 'view', 'rating'] = 2
events_final.loc[events_final['event'] == 'addtocart', 'rating'] = 3
events_final.loc[events_final['event'] == 'transaction', 'rating'] = 5

for index, row in events_final.iterrows():
    events_tupel.append((row['visitorid'], row['itemid'], row['rating']))



In [3]:
def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()
        

with open(data_folder+'/events_rating_tupel_1.json', 'w') as fp:
    json.dump(events_tupel, fp, default=myconverter, indent=2)

In [4]:
### load categories and items
category_json = data_folder+"cleaned_category.json"
cleaned_category = None
with open(category_json) as json_file:
    cleaned_category = json.load(json_file)
    
len(cleaned_category) 

1669

In [5]:
items_json = data_folder+"cleaned_items_indent.json"
cleaned_items = None
with open(items_json) as json_file:
    cleaned_items = json.load(json_file)
    
len(cleaned_items)

417053

In [6]:
items_array = [] #cleaned_items.values()

for item_id in cleaned_items:
    item_obj = cleaned_items[item_id]
    item = dict()
    categoryid = "_"+str(item_obj['property_categoryid'])
    #print(cleaned_category[categoryid])
    #print(categoryid)
    category = None 
    if categoryid in cleaned_category:
        category = cleaned_category[categoryid]
        #print(category)
    #item = cleaned_items[item_id]
    item['item_id'] = item_id
    item['property_790'] = item_obj['property_790']
    if category is not None:
        paths = category['path'].split('/')
        size = len(paths)
        item['cat_1'] = "_"+paths[0] if size > 0 else None  
        item['cat_2'] = "_"+paths[1] if size > 1 else None
        item['cat_3'] = "_"+paths[2] if size > 2 else None
        
    items_array.append(item)
    #break


print(len(items_array))
#print(items_array[0])
items = pd.DataFrame(items_array)
#items = items[items.columns.drop(list(items.filter(regex='_ts_')))]


417053


In [7]:
items.head(10)

,item_id,property_790,cat_1,cat_2,cat_3
0,460429,378360.0,_395,_1278,_1338
1,206783,179760.0,_1600,_872,_1313
2,395014,172200.0,_140,_1519,_1329
3,59481,12720.0,_395,_1257,_256
4,156781,32520.0,_1532,_500,_1012
5,285026,70560.0,_1224,_871,_624
6,89534,20280.0,_431,_362,_1577
7,264312,29040.0,_140,_61,_323
8,229370,889056.0,_250,_1335,_485
9,98113,108240.0,_1532,_500,_48


In [8]:
events = pd.read_csv(data_folder+"events.csv", header= 0)
events_reduced = events.drop_duplicates()
events_reduced = events_reduced.drop_duplicates(['visitorid', 'event', 'itemid']).copy()
events_final = events_reduced.sort_values(by='timestamp', ascending=True).copy()
events_final.loc[events_final['event'] == 'view', 'rating'] = 2
events_final.loc[events_final['event'] == 'addtocart', 'rating'] = 3
events_final.loc[events_final['event'] == 'transaction', 'rating'] = 5


In [20]:
events_tupel = []
with open(data_folder+'/events_rating_tupel_1.json', 'r') as fp:
    events_tupel = json.load(fp)

In [21]:
#### buillding user-item matrix
from lightfm.data import Dataset

feature_keys = ['cat_1', 'cat_2', 'cat_3', 'property_790'];

dataset = Dataset(user_identity_features=False, item_identity_features=True)
dataset.fit(events_final['visitorid'].to_numpy(), events_final['itemid'].to_numpy(), item_features=feature_keys)

dataset.interactions_shape()

(1407580, 235061)

In [22]:

start_index = int(0.2*len(events_tupel))
end_index = int(0.25*len(events_tupel))
events_tupel = events_tupel[start_index:end_index]

total_events = len(events_tupel)

train_amt = int(0.9*total_events)
events_train = events_tupel[:train_amt]
events_test = events_tupel[(train_amt+1):]

print(len(events_train))
print(len(events_test))

96533
10725


In [23]:
(interactions_train, weights_train) = dataset.build_interactions(events_train)
(interactions_test, weights_test) = dataset.build_interactions(events_test)




In [24]:
#### build features
item_features_tupel = []
items_missing = []
(item_mapping, user_mapping, item_features_mapping, user_features_mapping) = dataset.mapping()
for index, row in items.iterrows():
    item = row['item_id']
    row_features = []
    for attr in feature_keys:
        row_features.append(row[attr])
        
    if item not in item_mapping:
        items_missing.append([item, row_features])
    else:
        item_features_tupel.append([item, row_features])
        
item_features = dataset.build_item_features(item_features_tupel)


In [25]:

from lightfm import LightFM

model_bpr = LightFM(loss='bpr')
%time model_bpr.fit(interactions_train, item_features=item_features, epochs=4)

Wall time: 1.94 s


In [26]:

model_wrp = LightFM(loss='warp')
%time model_wrp.fit(interactions_train, item_features=item_features, epochs=4)

Wall time: 2.11 s


In [27]:
from lightfm.evaluation import auc_score

# Compute and print the AUC score
%time train_auc = auc_score(model_bpr, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Wall time: 16min 33s
Collaborative filtering train AUC: 0.53377795



Wall time: 16min 33s
Collaborative filtering train AUC: 0.53377795


In [28]:

# Compute and print the AUC score
%time train_auc = auc_score(model_wrp, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

#Wall time: 16min 17s
# Collaborative filtering train AUC: 0.95790994

Wall time: 16min 17s
Collaborative filtering train AUC: 0.95790994


Wall time: 17min 21s
Collaborative filtering train AUC: 0.9586296

In [29]:

%time train_auc = auc_score(model_bpr, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)
#Wall time: 4min 48s
# Collaborative filtering train AUC: 0.5048117

Wall time: 4min 48s
Collaborative filtering train AUC: 0.5048117


In [30]:

%time train_auc = auc_score(model_wrp, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

#Wall time: 1min 45s
#Collaborative filtering train AUC: 0.75068545

Wall time: 1min 45s
Collaborative filtering train AUC: 0.75068545


In [31]:
from lightfm.evaluation import precision_at_k as precision

# Compute and print the AUC score
%time train_auc = precision(model_bpr, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train Precision: %s' % train_auc)


%time train_auc = precision(model_bpr, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering test Precision: %s' % train_auc)

#Wall time: 23min 36s
#Collaborative filtering train Precision: 0.00016036959
#Wall time: 2min 51s
#Collaborative filtering test Precision: 6.333925e-05

Wall time: 23min 36s
Collaborative filtering train Precision: 0.00016036959
Wall time: 2min 51s
Collaborative filtering test Precision: 6.333925e-05


In [32]:

%time train_auc = precision(model_wrp, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train Precision: %s' % train_auc)


%time train_auc = precision(model_wrp, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering test Precision: %s' % train_auc)

#Wall time: 1h 6min 5s
#Collaborative filtering train Precision: 0.012636829
#Wall time: 3min 7s
#Collaborative filtering test Precision: 0.0010767672

Wall time: 1h 6min 5s
Collaborative filtering train Precision: 0.012636829
Wall time: 3min 7s
Collaborative filtering test Precision: 0.0010767672


In [34]:
from lightfm.evaluation import recall_at_k as recall

%time train_auc = recall(model_bpr, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train Recall: %s' % train_auc)


%time train_auc = recall(model_bpr, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering test Recall: %s' % train_auc)

#Wall time: 30min 6s
#Collaborative filtering train Recall: 0.0010554820614362384
#Wall time: 1min 48s
#Collaborative filtering test Recall: 0.00032336351392796655

Wall time: 30min 6s
Collaborative filtering train Recall: 0.0010554820614362384
Wall time: 1min 48s
Collaborative filtering test Recall: 0.00032336351392796655


In [35]:

%time train_auc = recall(model_wrp, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train Recall: %s' % train_auc)


%time train_auc = recall(model_wrp, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering test Recall: %s' % train_auc)

#Wall time: 13min 43s
#Collaborative filtering train Recall: 0.09859760712373451
#Wall time: 1min 31s
#Collaborative filtering test Recall: 0.008779287437284589

Wall time: 13min 43s
Collaborative filtering train Recall: 0.09859760712373451
Wall time: 1min 31s
Collaborative filtering test Recall: 0.008779287437284589


In [107]:

#print(interactions_test.tocoo())
user_purchases = dict()
#for row in interactions_test.tocsc():
    
#for row in weights_test:
    #(user, item) = row[0]
    
    #rating = row[1]
    #if rating == 5:
    #    if not user in user_purchases:
    #        user_purchases[user] = []
    #    user_purchases[user].append(item)
        
    
##print(user_purchases)
    #
#print(["377133"].shape)

def retriev_top_ten(model, user, items, predictions, item_features):
    predictions = model.predict(user, items, item_features=item_features)
    item_dict = dict()
    for key in range(0, len(predictions)):
        pred = predictions[key]
        item = items[key]
        item_dict[str(item)] = pred
        
    #print(item_dict)
    sort_items = sorted(item_dict.items(), key=lambda x: x[1], reverse=True)
    #tt = (sorted(item_dict.items(), key = lambda kv:(kv[1], kv[0])))  
    result = dict()
    #sort_keys = sort_items.keys()
    sort_keys = sort_items[:10]
    #print(sort_keys)
    for i in range(0, len(sort_keys)):
        (_id, _score) = sort_keys[i]
        result[_id] = {'item':_id, "score":_score}
        
    return result
        
n_items = np.arange(235061)
result = retriev_top_ten(model_wrp, 88, n_items, predictions, item_features=item_features)
print(result)  


#model_wrp.predict("438841", ["377133"], item_features=item_features)


[('40', 1.6951541900634766), ('2929', 1.6887784004211426), ('104', 1.6225441694259644), ('411', 1.6115334033966064), ('1132', 1.575950026512146), ('2256', 1.5595812797546387), ('273', 1.5511711835861206), ('2012', 1.5461903810501099), ('55589', 1.488364815711975), ('6627', 1.4880174398422241)]
{'40': {'item': '40', 'score': 1.6951541900634766}, '2929': {'item': '2929', 'score': 1.6887784004211426}, '104': {'item': '104', 'score': 1.6225441694259644}, '411': {'item': '411', 'score': 1.6115334033966064}, '1132': {'item': '1132', 'score': 1.575950026512146}, '2256': {'item': '2256', 'score': 1.5595812797546387}, '273': {'item': '273', 'score': 1.5511711835861206}, '2012': {'item': '2012', 'score': 1.5461903810501099}, '55589': {'item': '55589', 'score': 1.488364815711975}, '6627': {'item': '6627', 'score': 1.4880174398422241}}
